In [ ]:
import ee
from datetime import datetime
from dateutil.relativedelta import *
from helpers import index

ee.Initialize()

### Helper functions

In [ ]:
def renameBands(bands: list, sufix: str) -> list:
    mapBands = map(lambda band: band + '_' + sufix, bands)
    return list(mapBands)

def removeCloud(image: ee.image.Image) -> ee.image.Image:
    cloudThreshould = image.select('cloud').lt(0.23)
    shadeThreshould = image.select('shade').lt(0.15)

    return image.mask(cloudThreshould.Or(shadeThreshould))

def calculateArea(image: ee.image.Image, region: ee.feature.Feature):
    # area in ha
    pxArea = ee.Image.pixelArea().divide(10000)

    area = pxArea.multiply(image).reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'scale': 10,
        'geometry': region.geometry()
    }).get('area')

    return area



### Config variables

In [ ]:
# date params
YEAR = '2021'
MONTH = '08'
VERSION = '3'



MONTHS = [
    #'01', 
    #'02',
    #'03',
    #'04',
    #'05',
    #'06',
    #'07',
    '08',
    #'09',
    #'10',
    #'11',
    #'12'
]

BANDS = [
    'B2', 
    'B3', 
    'B4', 
    'B8', 
    'B11', 
    'B12'
]

NEW_BAND_NAMES = [
    'blue',
    'green',
    'red',
    'nir',
    'swir1',
    'swir2'
]

FEATURE_STACK = [
    'blue',
    'green',
    'red',
    'nir',
    'swir1',
    'swir2',
    'gv',
    'npv',
    'soil',
    'cloud',
    'shade',
    'gvs',
    'ndfi'
]

ASSET_DATASET_RAW = 'users/jailson/sad_ai/classificados_raw/alertas_raw_classificados_2021_08'
ASSET_OUTPUT = 'users/jailson/sad_ai/sad_chips'

BANDS_T0 = renameBands(FEATURE_STACK, 't0')
BANDS_T1 = renameBands(FEATURE_STACK, 't1')
BANDS_DIFF = renameBands(FEATURE_STACK, 'diff')

CLOUD_COVER = 30


### Check alread loaded images

In [ ]:

alreadyProcessed = (ee.ImageCollection(ASSET_OUTPUT))

listIdsLoaded = alreadyProcessed.reduceColumns(ee.Reducer.toList(), ['id_gee'])\
    .get('list').getInfo()


### Get input data

In [ ]:
dataset = ee.FeatureCollection(ASSET_DATASET_RAW)\
    .filter('sensor == "S2"')\
    .filter('validado == 0')


listIds = dataset.reduceColumns(ee.Reducer.toList(), ['system:index'])\
    .get('list').getInfo()

listIds = list(set(listIds)-set(listIdsLoaded))

len(listIds)

### Iterate over list ids

In [6]:
import pprint


for month in MONTHS:

    CURRENT_DATE = datetime.strptime('2021-{}-01'.format(month), '%Y-%m-%d')

    if month in ['01', '02', '03', '04', '05', '06', '07', '08']:
        T1_PERIOD = [
            CURRENT_DATE + relativedelta(months=-2), # START
            CURRENT_DATE + relativedelta(months=+1), # END
        ]

        T0_PERIOD = [
            CURRENT_DATE + relativedelta(months=-8), # START
            T1_PERIOD[0] + relativedelta(day=0), # END
        ]

    else:
        T0_PERIOD = [
            CURRENT_DATE + relativedelta(months=-1),
            CURRENT_DATE + relativedelta(days=-1)
        ]
        T1_PERIOD = [
            CURRENT_DATE,
            CURRENT_DATE + relativedelta(days=+29)
        ]

    T0_PERIOD[0] = T0_PERIOD[0].strftime('%Y-%m-%d')
    T0_PERIOD[1] = T0_PERIOD[1].strftime('%Y-%m-%d')
    T1_PERIOD[0] = T1_PERIOD[0].strftime('%Y-%m-%d')
    T1_PERIOD[1] = T1_PERIOD[1].strftime('%Y-%m-%d')


    for idAlert in listIds:

        alertFeat = ee.Feature(dataset.filter('system:index == "' + idAlert + '"').first()).set('value', 1)

        #     
        # alertBound = ee.Algorithms.If(
        #     ee.Number(alertFeat.get('areakm2')).lt(0.0625), # condition
        #     alertFeat.buffer(130).bounds(), # if true, for small size polygons
        #     alertFeat.bounds() # if false
        # ).getInfo()["geometry"]["coordinates"]

        alertBound = alertFeat.buffer(130).bounds().getInfo()["geometry"]["coordinates"] # if true, for small size polygons


        alertBoundFeat = ee.Feature(ee.Geometry.MultiPolygon(alertBound)).set('value', 1)

        # transform bound geometry to bound img
        alertBoundImg = ee.FeatureCollection(alertBoundFeat)\
            .reduceToImage(['value'], ee.Reducer.first())

        # transform only alert polygon to img
        alertImg = ee.FeatureCollection(alertFeat).reduceToImage(['value'], ee.Reducer.first())

        # convert to binary label
        alertImg = alertBoundImg.where(alertImg.eq(1), 0).remap([0, 1], [1, 0])\
            .rename('segment')


        collectionT0 = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
                    .filterBounds(alertFeat.geometry())
                    .filterDate(T0_PERIOD[0], T0_PERIOD[1])
                    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_COVER))
                    .select(BANDS, NEW_BAND_NAMES)
                    .map(lambda image: image.divide(10000).copyProperties(image))
                    .map(index.getFractions)
                    .map(index.getNdfi)
                    .map(removeCloud)
                    .sort('system:time_start'))

        collectionT1 = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
                    .filterBounds(alertFeat.geometry())
                    .filterDate(T1_PERIOD[0], T1_PERIOD[1])
                    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_COVER))
                    .select(BANDS, NEW_BAND_NAMES)
                    .map(lambda image: image.divide(10000).copyProperties(image))
                    .map(index.getFractions)
                    .map(index.getNdfi)
                    .map(removeCloud)
                    .sort('system:time_start'))

        imageT0 = ee.Image(collectionT0.mosaic()).rename(BANDS_T0)
        imageT1 = ee.Image(collectionT1.mosaic()).rename(BANDS_T1)
        diff = ee.Image(imageT0.subtract(imageT1)).rename(BANDS_DIFF)

        imageOutput = imageT0\
                .addBands(imageT1)\
                .addBands(diff)\
                .addBands(alertImg)\
                .clip(alertBoundFeat)

        # calculate mask area
        cloudMaskT0 = imageOutput.unmask(100).eq(100).select('ndfi_t0')
        cloudMaskT1 = imageOutput.unmask(100).eq(100).select('ndfi_t1')

        areaMaskT0 = calculateArea(cloudMaskT0, alertBoundFeat)
        areaMaskT1 = calculateArea(cloudMaskT1, alertBoundFeat)
        boundArea = calculateArea(alertBoundImg, alertBoundFeat)

        areaMaskT0P = ee.Number(areaMaskT0).multiply(100).divide(ee.Number(boundArea))
        areaMaskT1P = ee.Number(areaMaskT1).multiply(100).divide(ee.Number(boundArea))

        imageOutput = imageOutput\
            .set('t0_period', ','.join(T0_PERIOD))\
            .set('t1_period', ','.join(T1_PERIOD))\
            .set('t0_cloudmask_area_ha', areaMaskT0)\
            .set('t0_cloudmask_area_perc', areaMaskT0P)\
            .set('t1_cloudmask_area_ha', areaMaskT1)\
            .set('t1_cloudmask_area_perc', areaMaskT1P)\
            .set('bound_area_ha', boundArea)\
            .set('version', int(VERSION))\
            .set('year', 2021)\
            .set('sensor', 'S2')\
            .set('id_gee', idAlert)\
            .set('src', 'IMAZON')\
            .set('status', alertFeat.get('validado'))\
            .set('area_alerta_km2', alertFeat.get('area'))

        region = alertBound
        
        name = idAlert + '_{}_{}_{}'.format(YEAR, MONTH, VERSION)


        task = ee.batch.Export.image.toAsset(
            image=imageOutput,
            description=name,
            assetId=ASSET_OUTPUT + '/' + name,
            region=region,
            scale=30,
            #maxPixels=1e+13
        )

        task.start()

        print(name)


KeyboardInterrupt: 